<a href="https://colab.research.google.com/github/bkricardo/attention_base_speech_to_text/blob/main/final_model_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import json
drive.mount("/content/gdrive",force_remount=True)
%cd ./gdrive/My Drive/11785-hw/hw4/data

Mounted at /content/gdrive
/content/gdrive/My Drive/11785-hw/hw4/data


In [ ]:
%cd /content

/content


In [ ]:
!pip install kaggle
!mkdir .kaggle

token = {"username":"bkyuanpeiw","key":"da89ec8ea1e4ac3d0df30a0597685d40"}
with open('/content/.kaggle/kaggle.json','w') as file:
    json.dump(token,file)

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}


In [ ]:
'''
@Author: yuanpei-wang
@Last Edit: 2021/03/28
'''

import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
import torch.optim as optim
import torch.nn.utils as utils
from torch.autograd import Variable
import seaborn as sns
import matplotlib.pyplot as plt
import time
import random
from torch.utils import data

from tqdm.notebook import tqdm as tq
from torch.nn.utils.rnn import *

cuda = torch.cuda.is_available()
print(cuda, sys.version)
device = torch.device("cuda" if cuda else "cpu")
np.random.seed(11785)
torch.manual_seed(11785)

LETTER_LIST = ['<sos>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', \
         'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-', "'", '.', '_', '+', ' ', '<eos>']

True 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


In [ ]:
def create_dictionaries(letter_list):
    '''
    Create dictionaries for letter2index and index2letter transformations
    '''
    letter2index = dict()
    index2letter = dict()
    for index, letter in enumerate(LETTER_LIST):
        letter2index[letter] = index
        index2letter[index] = letter

    return letter2index, index2letter


def transform_letter_to_index(raw_transcripts):
    '''
    Transforms text input to numerical input by converting each letter
    to its corresponding index from letter_list

    Args:
        raw_transcripts: Raw text transcripts with the shape of (N, )

    Return:
        transcripts: Converted index-format transcripts. This would be a list with a length of N
    '''
    index_list = []
    for sentence in raw_transcripts:
        letters = [letter2index['<sos>']]
        # print(sentence)
        for word in sentence:
            # print(word)
            new_word = word.decode('utf-8')

            for char in new_word:
                letters.append(letter2index[char])

            letters.append(letter2index[' '])
        letters.pop()
        letters.append(letter2index['<eos>'])
        index_list.append(letters)

    return index_list

# Create the letter2index and index2letter dictionary
letter2index, index2letter = create_dictionaries(LETTER_LIST)

In [ ]:
%cd ./gdrive/My Drive/11785-hw/hw4/data
test_data = np.load('test.npy', allow_pickle=True, encoding='bytes')

/content/gdrive/My Drive/11785-hw/hw4/data


In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        # For testing set, return only x
        if self.Y == None:
            return torch.tensor(self.X[index].astype(np.float32))
        # For training and validation set, return x and y
        else:
            return torch.tensor(self.X[index].astype(np.float32)), torch.tensor(self.Y[index])

def collate_test(data):
    """
    Return:
        pad_x: the padded x (testing speech data)
        x_len: the length of x
    """
    inputs = [x for x in data]
    inputs_pad = pad_sequence(inputs, batch_first=True) # dim (B, T, C) since batch_first is true, (T, B, C) if false
    inputs_lens = torch.LongTensor([len(x) for x in data])

    return inputs_pad,inputs_lens

In [ ]:
test_dataset = MyDataset(test_data,Y= None)
test_loader = data.DataLoader(
            dataset = test_dataset,
            batch_size = 32,
            num_workers = 4,
            shuffle = False,
            collate_fn = collate_test,
            pin_memory = True)

In [ ]:
class pBLSTM(nn.Module):
    '''
    Pyramidal BiLSTM
    Read paper and understand the concepts and then write your implementation here.
    '''
    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bidirectional=True, batch_first=True)

    def forward(self, x):

        x_padded, x_len = pad_packed_sequence(x, batch_first=True)  #(B,T ,hidden), unpack x

        x_len.to(device)

        # cut the x_padded if time length is odd
        x_padded = x_padded[:,:(x_padded.size(1) // 2) * 2, :]

        #concat 2 time step to 1 time step with 2 dimension concated, (B, T/2, hidden * 2)
        x_concat = x_padded.reshape(x_padded.size(0), x_padded.size(1) // 2, x_padded.size(2) * 2)
        x_len = x_len // 2

        x_packed = pack_padded_sequence(x_concat,lengths = x_len, enforce_sorted=False, batch_first=True)

        out = self.blstm(x_packed)[0]

        return out

In [ ]:
class Encoder(nn.Module):
    '''
    Encoder takes the utterances as inputs and returns the key, value and unpacked_x_len.
    Key and value are linear projections of the output from pBLSTM network for the laster.
    '''
    def __init__(self, input_dim, encoder_hidden_dim, key_value_size=128):
        super(Encoder, self).__init__()
        # The first LSTM at the very bottom
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=encoder_hidden_dim, bidirectional=True, batch_first=True)


        # TODO: Define the blocks of pBLSTMs
        # ...
        self.plstm = nn.Sequential(
            pBLSTM(encoder_hidden_dim*4, encoder_hidden_dim),
            pBLSTM(encoder_hidden_dim*4, encoder_hidden_dim),
            pBLSTM(encoder_hidden_dim*4, encoder_hidden_dim)
        )

        # The linear transformation for producing Key and Value for attention
        # Since you are using bidirectional LSTM, be careful about the size of hidden dimension
        self.key_network = nn.Linear(encoder_hidden_dim*2, key_value_size)
        self.value_network = nn.Linear(encoder_hidden_dim*2, key_value_size)

    def forward(self, x, x_len):
        # Pass through the first LSTM at the very bottom
        packed_sequence = rnn_utils.pack_padded_sequence(x, x_len, enforce_sorted=False, batch_first=True)
        out, _ = self.lstm(packed_sequence)


        # TODO: Pass through the pBLSTM blocks
        out = self.plstm(out)

        # Unpack the sequence and get the Key and Value for attention
        unpacked_out, unpacked_len = pad_packed_sequence(out,batch_first=True)

        key = self.key_network(unpacked_out)
        value = self.value_network(unpacked_out)

        # return key, value, unpacked_x_len
        return key, value, unpacked_len

In [ ]:
def plot_attention(attention):
    plt.clf()
    sns.heatmap(attention, cmap='GnBu')
    plt.show()

class Attention(nn.Module):
    '''
    Attention is calculated using key, value and query from Encoder and decoder.
    Below are the set of operations you need to perform for computing attention:
        energy = bmm(key, query)
        attention = softmax(energy)
        context = bmm(attention, value)
    '''

    def __init__(self):
        super(Attention, self).__init__()

    def forward(self, query, key, value, lengths):
        """
            :param query: (batch_size, hidden_size), decoder state of a single timestep
            :param key: (batch_size, max_len, key_value_size), encoded input sequences for key projection
            :param value: (batch_size, max_len, key_value_size), encoded input sequences for value projection
            :param lengths: (batch_size,), lengths of source sequences
            :returns: (batch_size, hidden_size) attended source context, and (batch_size, max_len) attention vectors
        """
        # Compute (batch_size, max_len) attention logits. "bmm" stands for "batch matrix multiplication".
        # Input shape of bmm:  (batch_szie, max_len, hidden_size), (batch_size, hidden_size, 1)
        # Output shape of bmm: (batch_size, max_len, 1)
        #hidden should = key_value_size
        energy = torch.bmm(key, query.unsqueeze(2)).squeeze(2) #(B,max_len, key_value_size) * (B, hidden_size,1) = (B, max_len,1) -> (B, max_len)

        # Create an (batch_size, max_len) boolean mask for all padding positions
        # Make use of broadcasting: (1, max_len), (batch_size, 1) -> (batch_size, max_len)
        mask = torch.arange(key.size(1)).unsqueeze(0) >= lengths.unsqueeze(1)
        mask = mask.to(device)

        # Set attention logits at padding positions to negative infinity.
        energy.masked_fill_(mask, -1e9) # (B, max_len)

        # Take softmax over the "source length" dimension.
        attention = nn.functional.softmax(energy, dim = 1)

        # Compute attention-weighted sum of context vectors
        # Input shape of bmm: (batch_size, 1, max_len), (batch_size, max_len, hidden_size)
        # Output shape of bmm: (batch_size, 1, hidden_size)
        context = torch.bmm(attention.unsqueeze(1), value).squeeze(1)   #(B,1, max_len) * (B,max_len,hiddn) = (B,1,hidden) -> (B,hidden)


        return context, attention

In [ ]:
class Decoder(nn.Module):
    '''
    As mentioned in a previous recitation, each forward call of decoder deals with just one time step.
    Thus we use LSTMCell instead of LSTM here.
    The output from the seond LSTMCell can be used as query for calculating attention.
    In place of value that we get from the attention, this can be replace by context we get from the attention.
    Methods like Gumble noise and teacher forcing can also be incorporated for improving the performance.
    '''
    def __init__(self, vocab_size, decoder_hidden_dim, embed_dim, key_value_size=128):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=letter2index['<eos>'])
        self.lstm1 = nn.LSTMCell(input_size=embed_dim + key_value_size, hidden_size=decoder_hidden_dim)
        self.lstm2 = nn.LSTMCell(input_size=decoder_hidden_dim, hidden_size=key_value_size)
        self.drop1 = LockedDropout(dropout = 0.2)
        self.drop2 = LockedDropout(dropout = 0.2)

        self.attention = Attention()
        self.vocab_size = vocab_size
        self.character_prob = nn.Linear(2 * key_value_size, vocab_size)
        self.key_value_size = key_value_size

        self.character_prob.weight=self.embedding.weight

    def forward(self, key, value, encoder_len, y=None, mode='train',teacher_f = 0.1):
        '''
        Args:
            key :(B, T, key_value_size) - Output of the Encoder Key projection layer
            value: (B, T, key_value_size) - Output of the Encoder Value projection layer
            y: (T, text_len) - Batch input of text with text_length
            mode: Train or eval mode
        Return:
            predictions: the character perdiction probability
        '''

        B, key_seq_max_len, key_value_size = key.shape


        if mode == 'train':
            max_len =  y.shape[1]
            char_embeddings = self.embedding(y)   # (B,T, hidden_size)
        else:
            max_len = 600

        # TODO: Create the attention mask here (outside the for loop rather than inside) to aviod repetition
        # Had already done in Attention()

        predictions = []
        prediction = torch.zeros(B, 1).to(device)
        hidden_states = [None, None]

        # TODO: Initialize the context. Be careful here
        context = torch.zeros(B,key_value_size).to(device)
        attentionPlot = []
        for i in range(max_len):
            if mode == 'train':
                # TODO: Implement (1) Teacher Forcing and (2) Gumble Noise techniques here
                teacher_force = True if np.random.random_sample() > teacher_f else False #set teacher forceing with 0.9

                if i == 0: #start character always <sos>
                        start_char = torch.zeros(B, dtype=torch.long).fill_(letter2index['<sos>']).to(device) #
                        char_embed = self.embedding(start_char)
                else:
                    if teacher_force:
                        char_embed = char_embeddings[:, i - 1, :]
                    else:
                        char_embed = self.embedding(prediction.argmax(dim=-1))
            else:
                if i ==0:
                    start_char = torch.zeros(B, dtype=torch.long).fill_(letter2index['<sos>']).to(device) #
                    char_embed = self.embedding(start_char)
                else:
                    char_embed = self.embedding(prediction.argmax(dim=-1))

            y_context = torch.cat([char_embed, context], dim=1)
            y_context = self.drop1(y_context.unsqueeze(1)).squeeze(1)
            hidden_states[0] = self.lstm1(y_context, hidden_states[0])

            lstm1_hidden = hidden_states[0][0]
            lstm1_hidden = self.drop1(lstm1_hidden.unsqueeze(1)).squeeze(1)
            hidden_states[1] = self.lstm2(lstm1_hidden, hidden_states[1])
            output = hidden_states[1][0]

            # TODO: Compute attention from the output of the second LSTM Cell

            context, attention = self.attention(output, key, value, encoder_len)


            # plot_attention(attention=attention)
            if mode == 'train':
                curr_attention = attention[0].detach().cpu()
                # print(curr_attention.shape)
                attentionPlot.append(curr_attention)

            output_context = torch.cat([output, context], dim=1)
            prediction = self.character_prob(output_context)
            predictions.append(prediction.unsqueeze(1))

        if mode == 'train':
            attention_map = torch.stack(attentionPlot, dim=1)
            return torch.cat(predictions, dim=1), attention_map
        else:
            return torch.cat(predictions, dim=1)

In [ ]:
class Seq2Seq(nn.Module):
    '''
    We train an end-to-end sequence to sequence model comprising of Encoder and Decoder.
    This is simply a wrapper "model" for your encoder and decoder.
    '''
    def __init__(self, input_dim, vocab_size, encoder_hidden_dim, decoder_hidden_dim, embed_dim, key_value_size=128):
        super(Seq2Seq,self).__init__()
        self.encoder = Encoder(input_dim, encoder_hidden_dim, key_value_size=key_value_size)
        self.decoder = Decoder(vocab_size, decoder_hidden_dim, embed_dim, key_value_size=key_value_size)

    def forward(self, x, x_len, y=None, mode='train',teacher_f = 0.1):
        key, value, encoder_len = self.encoder(x, x_len)
        if mode == 'train':
            predictions,att = self.decoder(key, value, encoder_len, y=y, mode=mode,teacher_f = teacher_f )
            return predictions,att
        else:
            predictions = self.decoder(key, value, encoder_len, y=y, mode=mode, teacher_f = teacher_f )
            return predictions

In [ ]:
#reference: # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
class LockedDropout(nn.Module):
    def __init__(self, dropout = 0.3):
        super().__init__()
        self.dropout = dropout

    def forward(self, x):
        if not self.training or not self.dropout:
            return x
        m = x.data.new(1, x.size(1), x.size(2)).bernoulli_(1 - self.dropout)
        mask = Variable(m, requires_grad=False) / (1 - self.dropout)
        mask = mask.expand_as(x)
        return mask * x

In [ ]:
def idx2string(index_list,stop_index):
    string_list = []

    for curr_time in index_list:

        curr_string = ""
        for idx in curr_time:
            if idx == stop_index:
                break
            else:
                curr_string += index2letter[idx]

        string_list.append(curr_string)

    return string_list

In [ ]:
model = Seq2Seq(input_dim=40,vocab_size=len(LETTER_LIST), encoder_hidden_dim= 256,decoder_hidden_dim= 512,
                embed_dim = 256, key_value_size=128)
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(40, 256, batch_first=True, bidirectional=True)
    (plstm): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
      (1): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
      (2): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
    )
    (key_network): Linear(in_features=512, out_features=128, bias=True)
    (value_network): Linear(in_features=512, out_features=128, bias=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(34, 256, padding_idx=33)
    (lstm1): LSTMCell(384, 512)
    (lstm2): LSTMCell(512, 128)
    (drop1): LockedDropout()
    (drop2): LockedDropout()
    (attention): Attention()
    (character_prob): Linear(in_features=256, out_features=34, bias=True)
  )
)

In [ ]:
checkpoint = torch.load('/content/gdrive/My Drive/11785-hw/hw4/model3/tfAdd3_12.pickle')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
predictions = np.array([])

start_time = time.time()
model.eval()
with torch.no_grad():
    for idx, (x, x_lens) in tq(enumerate(test_loader)):

        # 1) Set the inputs to the device.
        x= x.to(device)
        # 2) Pass your inputs, and length of speech into the model.
        prediction = model(x,x_lens,mode = 'val')
        idx_pred = prediction.argmax(-1).detach().cpu().numpy()
        string_pred = idx2string(idx_pred, letter2index['<eos>'])
        # print(string_pred.shape)
        predictions = np.concatenate((predictions,string_pred))

    end_time = time.time()
    print("predict time: ",end_time - start_time)


predict time:  31.224238634109497


In [ ]:
len(predictions)

2620

In [ ]:
with open("/content/gdrive/My Drive/11785-hw/hw4/first_try.csv",'w') as fh:
    fh.write("id,label\n") #don't type wrong or kaggle can't identify it
    for i in range(len(predictions)):
        fh.write("{:},{:}\n".format(i,predictions[i]))

In [ ]:
import pandas as pd
df = pd.read_csv("/content/gdrive/My Drive/11785-hw/hw4/first_try.csv")
df.head()

,id,label
0,0,yes tirday were trembling for he health for th...
1,1,we were more interested at a centinical condit...
2,2,they said sir but i must have money to do that...
3,3,aut of the gards my robbing the company i'll s...
4,4,but they could not in approve in a gace agains...


In [ ]:
%cd /content

/content


In [ ]:
%cd ./gdrive/My Drive/11785-hw/hw4/

/content/gdrive/My Drive/11785-hw/hw4


In [ ]:
!kaggle competitions submit -c 11785-homework-4-part-2-las-slack -f first_try.csv -m "gogogo"

100% 291k/291k [00:05<00:00, 50.3kB/s]
Successfully submitted to 11785 Homework 4 Part 2 : LAS - Slack